In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons

%matplotlib inline

In [ ]:
make_moons?

In [ ]:
X_train, y_train = make_moons(300, noise=.2, random_state=0)

In [ ]:
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=plt.cm.Spectral)

In [ ]:
class Layer:
    def __init__(self, n_input_dims, n_neurons, activation=None, weights=None, bias=None, random_state=0):
        np.random.seed(random_state)
        self.weights = weights if weights is not None else np.random.randn(
            n_input_dims, n_neurons) * np.sqrt(2 / n_input_dims)
        self.bias = bias if bias is not None else np.zeros(n_neurons)
        self.activation = activation
        self.A = None
        self.dA = None
        self.dZ = None
        self.dW = None
        self.db = None
        self.vdW = np.zeros_like(self.weights)
        self.vdb = np.zeros_like(self.bias)
        self.sdW = np.zeros_like(self.weights)
        self.sdb = np.zeros_like(self.bias)
        # 用来记录梯度的平方
        self.dW_2 =  np.zeros_like(self.weights)
        self.db_2 = np.zeros_like(self.bias)

    def activate(self, X):
        Z = np.dot(X, self.weights) + self.bias
        # 激活函数的输出
        self.A = self._apply_activation(Z)
        return self.A
    
    def _apply_activation(self, Z):
        # 激活函数
        if self.activation is None:
            return Z
        elif self.activation == 'relu':
            return np.maximum(0, Z)
        elif self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-Z))
        elif self.activation == 'tanh':
            return np.tanh(Z)
        else:
            return Z
    
    def derivative(self, A):
        # 激活函数的导数
        if self.activation is None:
            return np.ones_like(A)
        elif self.activation == 'relu':
            # return np.where(A <= 0, 0., 1.)
            d = np.array(A, copy=True)
            d[A <= 0] = 0.
            d[A > 0] = 1.
            return d 
        elif self.activation == 'sigmoid':
            return A  * (1 - A)
        elif self.activation == 'tanh':
            return 1 - A ** 2
        else:
            return A
    
    def __repr__(self):
        return f"weigths: {self.weights.shape}\n bias: {self.bias.shape}"

##  Gradient descent with Momentum

Requrie: 全局学习率 $\alpha$, 动量参数$\beta$  
Requrie: 初始参数 $\theta$, 初始速度$v=0$  
while 没有达到停止准则 do  
    
- 计算当前mini-batch的梯度: $g$   
- 计算速度更新: $v \leftarrow \beta v + (1-\beta)g$  
- 应用更新: $\theta \leftarrow \theta - \alpha v$  

end while


##  AdaGrad

Requrie: 全局学习率 $\alpha$  
Requrie: 初始参数 $\theta$  
Require: 小常数$\epsilon, 通常设为10^{-8}$(用来被小数除时的数值稳定)  
初始化梯度累计变量$r = 0$  
while 没有达到停止准则 do  

- 计算当前mini-batch的梯度: $g$   
- 累计平方梯度: $r \leftarrow  r + g\odot g$  
- 应用更新: $\theta \leftarrow \theta - \alpha \frac{1}{\sqrt{r}+\epsilon}\odot g$  

end while


##  RMSProp

Requrie: 全局学习率 $\alpha$, 衰减速率$\beta$    
Requrie: 初始参数 $\theta$  
Require: 小常数$\epsilon, 通常设为10^{-8}$(用来被小数除时的数值稳定)  
初始化累计变量$r = 0$  
while 没有达到停止准则 do  

- 计算当前mini-batch的梯度: $g$   
- 累计平方梯度: $r \leftarrow \beta r + (1-\beta)g\odot g$  
- 应用更新: $\theta \leftarrow \theta - \alpha \frac{g}{\sqrt{r}}$  

end while


$\beta$最常用的值是0.9

##  Adam

Requrie: 全局学习率 $\alpha$  
Requrie: 矩估计指数衰减速率$\beta_1, \beta_2 \in [0, 1)$(建议默认为0.9和0.999)    
Requrie: 初始参数 $\theta$  
Require: 小常数$\epsilon, 通常设为10^{-8}$(用来被小数除时的数值稳定)  
初始化参数$\theta$  
初始化一阶和二阶矩变量$s=0, r=0$
初始化同步时间$t=0$  
while 没有达到停止准则 do  

- 计算当前mini-batch的梯度: $g$ 
- $t \leftarrow t + 1$
- 更新有偏一阶矩估计: $s \leftarrow \beta_1 s + (1-\beta_1)g$
- 更新有偏二阶矩估计: $s \leftarrow \beta_2 r + (1-\beta_2)g \odot g$
- 修正一阶矩的偏差: $\hat s \leftarrow \frac{s}{1-\beta_1^t}$
- 修正一阶矩的偏差: $\hat r \leftarrow \frac{r}{1-\beta_2^t}$
- 应用更新: $\theta \leftarrow \theta - \alpha \frac{\hat s}{\sqrt{\hat r} + \epsilon}$  

end while

In [ ]:
class DeepNeuralNetwork:
    def __init__(self, lambda_=1e-3, lr=1e-3, max_epochs=10000, 
                 batch_size=64, optimizer='Adam', beta1=0.9,
                 beta2=0.999, epsilon=1e-8, verbose=1, random_state=0):
        self.layers = []
        self.lr = lr
        self.lambda_ = lambda_
        self.max_epochs = max_epochs
        self.batch_size = batch_size
        self.optimizer = optimizer
        self.beta1 = beta1
        self.beta2 = beta2
        self.verbose = verbose
        self.epsilon = epsilon
        self.seed = random_state
    
    def add_layer(self, layer):
        self.layers.append(layer)
    
    def feed_forward(self, X):
        for layer in self.layers:
            X = layer.activate(X)
        return X

    def loss(self, y_true, y_preds):
        # 交叉熵误差
        assert(y_true.shape == y_preds.shape)
        term = 0
        for layer in self.layers:
            term += np.sum(layer.weights ** 2)
        loss_ = - y_true* np.log(y_preds + self.epsilon) - (1 - y_true)*np.log(1 - y_preds + self.epsilon)  
        loss_ =  np.mean(loss_, axis=0) + self.lambda_ * term

        return np.squeeze(loss_)
    
    def backpropagation(self, X, y, t):
        # 反向传播算法 计算每一层的dW, db
        # 前向计算 得到输出值
        m = X.shape[0]  # batch size
        self.feed_forward(X)  # (m, 1)
        # print(out.shape)
        # loss_ = self.loss(y, out)
        for i in reversed(range(len(self.layers))):  # 从最后一层开始
            layer = self.layers[i]
            o_i = X if i == 0 else self.layers[i-1].A
            if layer == self.layers[-1]:  # 输出层
                # 使用 交叉熵 误差
                assert(y.shape == layer.A.shape)
                layer.dA = -y/(layer.A + self.epsilon) + (1-y) / (1-layer.A + self.epsilon)
                # layer.dA = -y / out + (1-y)/(1-out)  # dL/dA^K  # (m, 1)
            else:  # 隐藏层
                next_layer = self.layers[i + 1]
                layer.dA = next_layer.dZ @ next_layer.weights.T  # dL/dA^J (m, 1) (1, k)
                # dL/dZ^J (m, k) (m,k)
            layer.dZ = layer.dA * layer.derivative(layer.A)
            layer.dW =  o_i.T @ layer.dZ * 1 / m
            layer.db = np.sum(layer.dZ) * 1 / m
            w_term = self.lambda_ * layer.weights * 1 / m

            if self.optimizer == 'Momentum':
                layer.vdW = self.beta1 * layer.vdW + (1 - self.beta1) * layer.dW
                layer.vdb = self.beta1 * layer.vdb + (1 - self.beta1) * layer.db
                layer.weights -= self.lr * (layer.vdW + w_term)
                layer.bias -= self.lr * layer.vdb
            elif self.optimizer == 'RMSprop':
                layer.sdW = self.beta2 * layer.sdW + (1 - self.beta2) * layer.dW ** 2
                layer.sdb = self.beta2 * layer.sdb + (1 - self.beta2) * layer.db ** 2  
                layer.weights -= self.lr * (layer.dW / (layer.sdW ** 0.5 + self.epsilon) + w_term)
                layer.bias -= self.lr * layer.db / (layer.sdb ** 0.5 + self.epsilon)
            elif self.optimizer == 'AdaGrad':
                layer.dW_2 = layer.dW_2 + layer.dW ** 2
                layer.db_2 = layer.db_2 + layer.db ** 2
                layer.weights -= self.lr * (layer.dW / (layer.dW_2 ** 0.5 + self.epsilon) + w_term)
                layer.bias -= self.lr * layer.db / (layer.db_2 ** 0.5 + self.epsilon)
            elif self.optimizer == 'Adam':
                layer.vdW = self.beta1 * layer.vdW + (1 - self.beta1) * layer.dW
                layer.vdb = self.beta1 * layer.vdb + (1 - self.beta1) * layer.db
                # layer.vdW = vdW / (1 - self.beta1 ** t)
                # layer.vdb = vdb / (1 - self.beta1 ** t)
    
                layer.sdW = self.beta2 * layer.sdW + (1 - self.beta2) * layer.dW ** 2
                layer.sdb = self.beta2 * layer.sdb + (1 - self.beta2) * layer.db ** 2
                # layer.sdW = sdW / (1 - np.power(self.beta2, t))
                # layer.sdb = sdb / (1 -  np.power(self.beta2, t))

                # 改变计算顺序而得到提升效率 防止溢出
                lr = self.lr * (1 - self.beta2 ** t) ** 0.5 / (1 - self.beta1 ** t)
                layer.weights -= lr * layer.vdW / (layer.sdW ** 0.5 + self.epsilon) + self.lr * w_term
                layer.bias -= lr * layer.vdb / (layer.sdb ** 0.5 + self.epsilon)                       
            else:
                layer.weights -= self.lr * (layer.dW + w_term)
                layer.bias -=  self.lr * layer.db
         
    def preprocessing(self, X, y):
        y = y.reshape((-1, 1))
        self.seed += 1
        np.random.seed(self.seed)
        permutation = np.random.permutation(X.shape[0])
        shuffled_X = X[permutation]
        shuffled_y = y[permutation]
        
        return shuffled_X, shuffled_y
    
    def fit(self, X, y):
        y = y.reshape(-1, 1)
        cross_entropy = []
        # accuracy = []
        batch_size = self.batch_size
        m = X.shape[0]
        batches = m // batch_size if m % batch_size == 0 else m // batch_size + 1
        t = 0
        for epoch in range(self.max_epochs):
            # 每个epoch X,y 顺序都不一样
            
            shuffled_X, shuffled_y = self.preprocessing(X, y)
            # print(self.seed)
            for step in range(batches):
                
                batch_X = shuffled_X[batch_size*step: batch_size*(step+1)]
                batch_y = shuffled_y[batch_size*step: batch_size*(step+1)]
                # t 需要从1开始
                t += 1
                self.backpropagation(batch_X, batch_y, t)
            if epoch % 100 == 0:
                loss = self.loss(y, self.feed_forward(X))
                cross_entropy.append(loss)
            if epoch % 1000 == 0 and self.verbose:
#                 predict = self.predict(X_train)
#                 train_acc = np.sum(y_train.ravel()==predict) / len(predict)
                
#                 predict = self.predict(X_test)
#                 acc = np.sum(y_test.ravel()==predict) / len(predict)
#                 accuracy.append(acc)
                print(f'Epoch: {epoch}, cross_entropy: {loss}')
        return cross_entropy
    
    def predict(self, X):
        y_pred = self.feed_forward(X)
        y_pred = np.where(y_pred >=0.5, 1, 0)
        # out = np.argmax(y_pred, axis=1)
        return np.squeeze(y_pred)

In [ ]:
def plot_decision(model, X, y):
    x_max = X[:, 0].max() + 1
    x_min = X[:, 0].min() - 1
    y_max = X[:, 1].max() + 1
    y_min = X[:, 1].min() - 1
    xx, yy = np.mgrid[x_min:x_max:0.01, y_min:y_max:0.01]
    zz = np.c_[xx.ravel(), yy.ravel()]
    preds = model.predict(zz)
    plt.contourf(xx, yy, preds.reshape(xx.shape), cmap=plt.cm.Spectral)
    plt.ylabel('x2')
    plt.xlabel('x1')
    plt.scatter(X[:, 0], X[:, 1], c=np.squeeze(y), cmap=plt.cm.Spectral)
    plt.show()

### Mini-batch Gradient descent

In [ ]:
model_1 = DeepNeuralNetwork(lambda_=0, lr=0.0007, max_epochs=10000, optimizer='gd')
model_1.add_layer(Layer(X_train.shape[1], 5, activation='relu'))
model_1.add_layer(Layer(5, 2, activation='relu'))
model_1.add_layer(Layer(2, 1, activation='sigmoid'))

cross_entropy = model_1.fit(X_train, y_train)
preds = model_1.predict(X_train)
np.mean(preds == y_train)

In [ ]:
plt.plot(cross_entropy)
plt.show()
plt.title("Model with Gradient descent")

axes = plt.gca()
axes.set_xlim([-1.5, 2.5])
axes.set_ylim([-1, 1.5])
plot_decision(model_1, X_train, y_train)

### Mini-batch with Momentum mode

In [ ]:
model_2 = DeepNeuralNetwork(lambda_=0, lr=0.0007, max_epochs=10000, optimizer='Momentum')
model_2.add_layer(Layer(X_train.shape[1], 5, activation='relu'))
model_2.add_layer(Layer(5, 2, activation='relu'))
model_2.add_layer(Layer(2, 1, activation='sigmoid'))

cross_entropy = model_2.fit(X_train, y_train)
preds = model_2.predict(X_train)
np.mean(preds == y_train)

In [ ]:
plt.plot(cross_entropy)
plt.show()
plt.title("Model with Momentum optimization")

axes = plt.gca()
axes.set_xlim([-1.5, 2.5])
axes.set_ylim([-1, 1.5])
plot_decision(model_2, X_train, y_train)

### Mini-batch with RMSprop mode


In [ ]:
model_3 = DeepNeuralNetwork(lambda_=0, lr=0.0007, max_epochs=10000, optimizer='RMSprop')
model_3.add_layer(Layer(X_train.shape[1], 5, activation='relu'))
model_3.add_layer(Layer(5, 2, activation='relu'))
model_3.add_layer(Layer(2, 1, activation='sigmoid'))

cross_entropy = model_3.fit(X_train, y_train)
preds = model_3.predict(X_train)
np.mean(preds == y_train)

In [ ]:
plt.plot(cross_entropy)
plt.show()
plt.title("Model with RMSprop optimization")

axes = plt.gca()
axes.set_xlim([-1.5, 2.5])
axes.set_ylim([-1, 1.5])
plot_decision(model_3, X_train, y_train)

### Mini-batch with Adam mode

In [ ]:
t = 1
(1 - 0.999 ** t) ** 0.5 / (1 - 0.9 ** t)

In [ ]:
model_4 = DeepNeuralNetwork(lambda_=0, lr=0.0007, max_epochs=10000, optimizer='Adam')
model_4.add_layer(Layer(X_train.shape[1], 5, activation='relu'))
model_4.add_layer(Layer(5, 2, activation='relu'))
model_4.add_layer(Layer(2, 1, activation='sigmoid'))

cross_entropy = model_4.fit(X_train, y_train)
preds = model_4.predict(X_train)
np.mean(preds == y_train)

In [ ]:
plt.plot(cross_entropy)
plt.show()
plt.title("Model with Adam optimization")

axes = plt.gca()
axes.set_xlim([-1.5, 2.5])
axes.set_ylim([-1, 1.5])
plot_decision(model_4, X_train, y_train)

In [ ]:
model_5 = DeepNeuralNetwork(lambda_=0, lr=0.001, max_epochs=10000, optimizer='AdaGrad')
model_5.add_layer(Layer(X_train.shape[1], 5, activation='relu'))
model_5.add_layer(Layer(5, 2, activation='relu'))
model_5.add_layer(Layer(2, 1, activation='sigmoid'))

cross_entropy = model_5.fit(X_train, y_train)
preds = model_5.predict(X_train)
np.mean(preds == y_train)

In [ ]:
plt.plot(cross_entropy)
plt.show()
plt.title("Model with AdaGrad optimization")

axes = plt.gca()
axes.set_xlim([-1.5, 2.5])
axes.set_ylim([-1, 1.5])
plot_decision(model_5, X_train, y_train)

## 权重衰减

In [ ]:
class Network2(DeepNeuralNetwork):
    def __init__(self, lambda_=0.001, lr=0.001, max_epochs=10000, batch_size=64, optimizer='Adam', beta1=0.9,
                 beta2=0.999, epsilon=1e-08, verbose=1, random_state=0, decay_rate=0.5):
        super().__init__(lambda_=lambda_, lr=lr, max_epochs=max_epochs, batch_size=batch_size, optimizer=optimizer,
                         beta1=beta1, beta2=beta2, epsilon=epsilon, verbose=verbose, random_state=random_state)
        self.decay_rate = decay_rate
    def fit(self, X, y):
        y = y.reshape(-1, 1)
        cross_entropy = []
        # accuracy = []
        batch_size = self.batch_size
        m = X.shape[0]
        batches = m // batch_size if m % batch_size == 0 else m // batch_size + 1
        t = 0
        lr = self.lr
        for epoch in range(self.max_epochs):
            # 每个epoch X,y 顺序都不一样
            
            self.lr = lr * (1 / (1 + epoch * self.decay_rate))
            shuffled_X, shuffled_y = self.preprocessing(X, y)
            # print(self.seed)
            for step in range(batches):
                
                batch_X = shuffled_X[batch_size*step: batch_size*(step+1)]
                batch_y = shuffled_y[batch_size*step: batch_size*(step+1)]
                # t 需要从1开始
                t += 1
                self.backpropagation(batch_X, batch_y, t)
            if epoch % 100 == 0:
                loss = self.loss(y, self.feed_forward(X))
                cross_entropy.append(loss)
            if epoch % 1000 == 0 and self.verbose:
#                 predict = self.predict(X_train)
#                 train_acc = np.sum(y_train.ravel()==predict) / len(predict)
                
#                 predict = self.predict(X_test)
#                 acc = np.sum(y_test.ravel()==predict) / len(predict)
#                 accuracy.append(acc)          
                print(f'Epoch: {epoch}, cross_entropy: {loss}')
        return cross_entropy
    

In [ ]:
model_5 = Network2(lambda_=0, lr=0.0007, max_epochs=10000, optimizer='gd', decay_rate=0.0001)
model_5.add_layer(Layer(X_train.shape[1], 5, activation='relu'))
model_5.add_layer(Layer(5, 2, activation='relu'))
model_5.add_layer(Layer(2, 1, activation='sigmoid'))

cross_entropy = model_5.fit(X_train, y_train)
preds = model_5.predict(X_train)
np.mean(preds == y_train)